In [11]:
from pathlib import Path
from mri_data import utils
import nibabel as nib
import os
import json
import re
import statistics
import numpy as np
import pyperclip
import subprocess

In [3]:
dataroot = Path("/media/smbshare/3Tpioneer_bids")
training_work_home = Path("/home/srs-9/Projects/prl_project/training_work_dirs")
work_dir = training_work_home / "test_train0_swinunet" #! edit this
ensemble_dir = work_dir / "ensemble_output"
datalist_file_src = os.path.join(os.getcwd(), "datalist.json")
datalist_file = work_dir / "datalist.json"

In [22]:
with open(datalist_file, 'r') as f:
    datalist = json.load(f)

inferences = []
labels = []
subjects = []
for item in datalist['testing']:
    label = str(item['label'])
    assert os.path.exists(label)
    labels.append(label)
    subject = re.search(r"(sub-ms\d{4})", label)[1]
    subjects.append(subject)
    session = re.search(r"(ses-\d{8})", label)[1]
    inf = ensemble_dir / subject / session / "flair.phase.t1_ensemble.nii.gz"
    assert inf.exists()
    inferences.append(str(inf))

In [6]:
for sub in subjects:
    print(sub)

sub-ms1183
sub-ms1281
sub-ms2115
sub-ms1136
sub-ms1523
sub-ms1274
sub-ms2131
sub-ms1152
sub-ms2207
sub-ms1124
sub-ms1044
sub-ms1316
sub-ms1033
sub-ms1131
sub-ms2060
sub-ms1156


In [17]:
lesion_dice_scores = []
prl_dice_scores = []
for lab, inf in zip(labels, inferences):
    lab_data = nib.load(lab).get_fdata()
    inf_data = nib.load(inf).get_fdata()
    lesion_dice_scores.append(utils.dice_score(lab_data, inf_data, seg1_val=1, seg2_val=1))
    prl_dice_scores.append(utils.dice_score(lab_data, inf_data, seg1_val=2, seg2_val=2))

In [23]:
print("lesion dice: {:0.2f}".format(statistics.mean(lesion_dice_scores)))
print("PRL    dice: {:0.2f}".format(statistics.mean(prl_dice_scores)))

lesion dice: 0.82
PRL    dice: 0.47


In [25]:
def analyze_performance(lab, inf, val=1):
    TP = np.sum((lab == val) & (inf == val))
    FP = np.sum((lab != val) & (inf == val))
    TN = np.sum((lab != val) & (inf != val))
    FN = np.sum((lab == val) & (inf != val))

    sens = TP / (TP + FN)
    spec = FN / (TN + FP)
    ppv = TP / (TP + FP)
    npv = TN / (TN + FN)

    return {'sens': sens, 'spec': spec, 'ppv': ppv, 'npv': npv}

In [29]:
performance = []
for lab, inf in zip(labels, inferences):
    lab_data = nib.load(lab).get_fdata()
    inf_data = nib.load(inf).get_fdata()
    performance.append(analyze_performance(lab_data, inf_data, val=2))

In [23]:
subject = 1183
ind = subjects.index(f"sub-ms{subject}")
ind = 5
ground_truth = labels[ind]
inf = inferences[ind]
image_dir = Path(ground_truth).parent.parent

images = ["flair.nii.gz", "phase.nii.gz", "t1.nii.gz"]
image_paths = [image_dir / im for im in images]

label_paths = [inf, ground_truth]

cmd = utils.open_itksnap_workspace_cmd(image_paths, label_paths, win=False)
print(cmd)
pyperclip.copy(cmd)
subprocess.Popen(cmd.split(" "))

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1274/ses-20180504/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1274/ses-20180504/phase.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1274/ses-20180504/t1.nii.gz -s /home/srs-9/Projects/prl_project/training_work_dirs/test_train0_swinunet/ensemble_output/sub-ms1274/ses-20180504/flair.phase.t1_ensemble.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1274/ses-20180504/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz


<Popen: returncode: None args: ['itksnap', '-g', '/media/smbshare/3Tpioneer_...>

Launching ITK-SNAP


Return code : 0


In [40]:
performance[5]

{'sens': 0.08022239872915012,
 'spec': 5.3614236126077904e-05,
 'ppv': 0.36330935251798563,
 'npv': 0.9999463881988851}

In [41]:
labels[5]

'/media/smbshare/3Tpioneer_bids/sub-ms1274/ses-20180504/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz'

In [39]:
labels

['/media/smbshare/3Tpioneer_bids/sub-ms1183/ses-20170328/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms1281/ses-20180705/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms2115/ses-20180530/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms1136/ses-20171109/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms1523/ses-20170422/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms1274/ses-20180504/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms2131/ses-20190117/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms2207/ses-20180717/lesion.t3m20/lesion_and_prl_labels_tmp.nii.gz',
 '/media/smbshare/3Tpioneer_bids/sub-ms1124/ses-2016092